In [1]:
import pytest
from pydantic import ValidationError

def test_valid_tracking_number():
    tracking = TrackingNumber(value='TH1234567890')
    assert tracking.value == 'TH1234567890'

def test_empty_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value='')

def test_invalid_type_tracking_number():
    with pytest.raises(ValidationError):
        TrackingNumber(value=12345678)

In [2]:
from pydantic import BaseModel, Field, field_validator

class TrackingNumber(BaseModel):

    value: str = Field(..., min_length=1, json_schema_extra={'strip_whitespace': True})

    @field_validator('value')
    def check_length(cls, v):
        if len(v) < 5:
            raise ValueError('Tracking Number สั้นเกินไป')
        return v.strip()
    


if __name__ == "__main__":
    good_track = TrackingNumber(value='TH1234567890')
    print(f'สำเร็จ: ได้รับพัสดุ {good_track.value}')

    try:
        bad_track = TrackingNumber(value='TH12')
    except ValueError as e:
        print(f'โดนดัก error {e}')



สำเร็จ: ได้รับพัสดุ TH1234567890
โดนดัก error 1 validation error for TrackingNumber
value
  Value error, Tracking Number สั้นเกินไป [type=value_error, input_value='TH12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [3]:
def test_valid_money_creation():
    m = Money(amount=886.26, currency='THB')
    assert m.amount == 886.26
    assert m.currency == 'THB'

def test_money_negative_amount():
    with pytest.raises(ValidationError):
        Money(amount=-100.0, currency='THB')

def test_money_invalid_currency_format():
    with pytest.raises(ValidationError):
        Money(amount=100.0, currenry='THAI_BAHT')


In [11]:
class Money(BaseModel):
    amount: float = Field(..., ge=0)
    currency: str = Field(..., min_length=3, max_length=3)

    @field_validator('currency')
    def currency_must_be_uppercase(cls, v):
        return v.upper()
    
    def __str__(self):
        return f'{self.amount:,.2f} {self.currency}'


if __name__ == "__main__":
    try:
        # 1. สร้างเงินจากข้อมูลจริงของป๋า
        claim_money = Money(amount=1477.50, currency="thb")
        print(f"✅ สร้างเงินสำเร็จ: {claim_money}") # ผลลัพธ์จะเป็น 1,477.50 THB (เพราะมี __str__)

        # 2. ลองแกล้งใส่เงินติดลบ
        fake_money = Money(amount=-50, currency="USD")
    except Exception as e:
        print(f"❌ ระบบดักจับเงินติดลบได้: {e}")
    

✅ สร้างเงินสำเร็จ: 1,477.50 THB
❌ ระบบดักจับเงินติดลบได้: 1 validation error for Money
amount
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-50, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal


In [9]:
def test_claim_ticket_creation():
    tid = TicketId(value='CMP-1001')
    tn = TrackingNumber(value='TH1234567890')
    price = Money(amount=886.26, currency='THB')

    ticket = ClaimTicket(
        ticket_id=tid,
        tracking_number=tn,
        compensation_amount=price
    )

    assert ticket.ticket_id.value == 'CMP-1001'
    assert ticket.version == 1

In [14]:
class TicketId(BaseModel):
    value: str = Field(..., min_length=1)


class ClaimTicket(BaseModel):
    ticket_id: TicketId
    tracking_number: TrackingNumber
    compensation_amount:Money

    version: int = Field(default=1)

    def update_compensation(self, new_money: Money):
        self.compensation_amount = new_money
        self.version += 1
        print(f' อัพเดทยอดเงินเป็น {new_money} (Version: {self.version})')


if __name__ == '__main__':
    try:
            
        my_ticket = ClaimTicket(
            ticket_id=TicketId(value='CMP-1001'),
            tracking_number=TrackingNumber(value='TH1234567890'),
            compensation_amount=Money(amount=886.26, currency='THB')
        )
        print(f'ใบเคลมลูกค้าถูกสร้าง: {my_ticket.ticket_id.value}')
        print(f' ยอดเดิม: {my_ticket.compensation_amount}')

        new_price = Money(amount=950.00, currency='THB')
        my_ticket.update_compensation(new_price)
    except Exception as e:
        print(f'{e}') 



ใบเคลมลูกค้าถูกสร้าง: CMP-1001
 ยอดเดิม: 886.26 THB
 อัพเดทยอดเงินเป็น 950.00 THB (Version: 2)


In [17]:
def test_add_mutltiple_tickets_to_seme_case():
    tn = TrackingNumber(value='TH1234567888')

    claim_case = ClaimCase(tracking_number=tn)

    ticket1 = ClaimTicket(
        ticket_id=TicketId(value='CMP-1001'),
        tracking_number=tn,
        compensation_amount=Money(amount=500, currency='THB')
    )
    
    ticket2 = ClaimTicket(
        ticket_id=TicketId(value='CMP-1002'),
        tracking_number=tn,
        compensation_amount=Money(amount=300, currency='THB' )
    )

    claim_case.add_ticket(ticket1)
    claim_case.add_ticket(ticket2)

    assert len(claim_case.tickets) == 2
    assert claim_case.total_compensation.amount == 800.0

In [ ]:
class ClaimCase(BaseModel):
    """
    Aggregate Root: ตระกร้าคุมใบเคลมของสินค้า 1 ชิ้น
    """
    tracking_number: TrackingNumber
    tickets: list[ClaimTicket] = Field(default_factory=list)

    total_compensation: Money = Field(
        default_factory=lambda: Money(amount=0, currency='THB')
    )

    def add_ticket(self, ticket: ClaimTicket):
        """
        กฎธุรกิจ: 
        1. ตรวจสอบว่า Tracking ตรงกันไหม
        2. เพิ่มลง List
        3. อัปเดตยอดเงินรวม
        """
        if ticket.tracking_number.value != self.tracking_number.value:
            raise ValueError('ใบเคลมนี้คนละเลข Tracking กันนะ')
        
        self.tickets.append(ticket)

        new_amount = self.total_compensation.amount + ticket.compensation_amount.amount
        self.total_compensation = Money(amount=new_amount, currency='THB')

        print(f'เพิ่ม {ticket.ticket_id.value} สำเร็จ!')
        print(f'ยอยรวมเคสนี้: {self.total_compensation}')


if __name__ == '__main__':
    tn = TrackingNumber(value='TH1234567890')
    my_case = ClaimCase(tracking_number=tn)

    t1 = ClaimTicket(ticket_id=TicketId(value='CMP-01'), tracking_number=tn,
                   compensation_amount=Money(amount=100, currency='THB'))
    
    my_case.add_ticket(t1)

    t2 = ClaimTicket(ticket_id=TicketId(value='CMP-02'), tracking_number=tn,
                    compensation_amount=Money(amount=200, currency='THB'))
    my_case.add_ticket(t2)

เพิ่ม CMP-01 สำเร็จ!
ยอยรวมเคสนี้: 100.00 THB
เพิ่ม CMP-02 สำเร็จ!
ยอยรวมเคสนี้: 300.00 THB
